In [1]:
import os
import glob
import pandas as pd
import matplotlib
import numpy as np
from numpy import NaN
import matplotlib.pyplot as plt

In [2]:
# Combine all the CSV files
path = r'../data'
all_csv = glob.glob(path + "/*.csv")

temp = []
for csv in all_csv:
    df = pd.read_csv(csv, index_col=0, header=0, thousands=',')
    temp.append(df)

df = pd.concat(temp, axis=0, ignore_index=True)
df.head()

,Service,Type,Rating,Hires,Price,Zip Code
0,Second Growth Homes,123,NaN,NaN,NaN,99801
1,All-Pro Contractors,129,5.0,276.0,NaN,48226
2,Foster Exp.,129,5.0,49.0,42/hour,48226
3,B-Major Carpentry,129,4.5,12.0,75/hour,48226
4,Madison Property Services,129,4.5,12.0,80/hour,48226


In [3]:
df.shape

(3577, 6)

In [4]:
# Check and fix dtype
df.dtypes

Service      object
Type          int64
Rating      float64
Hires       float64
Price        object
Zip Code      int64
dtype: object

In [5]:
# See some prices values
df['Price'].unique()[:19]

# Need to fix that hour thing

array([nan, '42/hour', '75/hour', '80/hour', 29.0, 35.0, 45.0, 40.0, 20.0,
       30.0, 36.0, '49/on-site', 100.0, 120.0, 130.0, 125.0, 50.0, 33.0,
       105.0], dtype=object)

In [6]:
# Convert price to string
df['Price'] = df['Price'].astype(str)

# Removed the hour into another column
tempPrice = []
tempPriceType = []
for i in df['Price']:
    x = i.split('/')
    tempPrice.append(x[0])
    if i.find('hour') > 0:
        x = 'hourly'
    else:
        x = NaN
    tempPriceType.append(x)

# Add it to dataframe and check
df['Price'] = tempPrice
df['Price Type'] = tempPriceType

# Convert price column to float
df['Price'] = df['Price'].str.replace(",","").astype(float)

In [7]:
# Double check data types
df.dtypes

Service        object
Type            int64
Rating        float64
Hires         float64
Price         float64
Zip Code        int64
Price Type     object
dtype: object

In [8]:
# Add the service type from database
serviceDf = pd.read_csv('../src/all_services.csv')

dfMerge = df.rename(columns={"Type":"ID"})
dfMerge = pd.merge(dfMerge, serviceDf, left_on='ID', right_on='ID', how='left').drop('ID', axis=1)
df = dfMerge

In [9]:
# See shape
df.shape

(3577, 9)

In [10]:
# Check for duplicates
dfDuplicate = df[df.duplicated()]
dfDuplicate

# Deal with duplicates accordingly if there are any

,Service,Rating,Hires,Price,Zip Code,Price Type,Services,Type,Segments


In [11]:
# See overview of data
df.describe()

,Rating,Hires,Price,Zip Code
count,3293.000000,2369.000000,1646.000000,3577.000000
mean,4.779532,56.398058,98.288578,56919.003075
std,0.439865,143.465028,223.657693,28776.028946
min,1.000000,2.000000,1.000000,10007.000000
25%,4.500000,6.000000,50.000000,30303.000000
50%,5.000000,16.000000,82.000000,55404.000000
75%,5.000000,46.000000,110.000000,84111.000000
max,5.000000,2040.000000,6000.000000,99801.000000


In [19]:
# See basic count
servicesCount = df.groupby("Services")["Services"].count().to_frame(name="Count").reset_index()
servicesCount = servicesCount.sort_values(by="Count", ascending=False)
servicesCount["Contribution"] = servicesCount["Count"] / servicesCount["Count"].sum()
servicesCount

,Services,Count,Contribution
5,Roof Installation or Replacement,956,0.267263
3,Insulation Installation or Upgrade,828,0.231479
2,Hot Tub and Spa Repair,596,0.166620
4,Lawn Mower Repair,445,0.124406
0,General Carpentry,396,0.110707
1,Gutter Repair,356,0.099525


In [13]:
# Get implied revenue
df["Implied Revenue"] = df["Price"] * df["Hires"]

# Get sum of implied revenue by segment
dfRevenuBySegment = df.groupby("Segments")["Implied Revenue"].sum().to_frame(name="Implied Revenue").reset_index()
dfRevenuBySegment["Contribution"] = dfRevenuBySegment["Implied Revenue"] / dfRevenuBySegment["Implied Revenue"].sum()

# Get sum of implied revenue by services
dfRevenuByServices = df.groupby("Services")["Implied Revenue"].sum().to_frame(name="Implied Revenue").reset_index()
dfRevenuByServices["Contribution"] = dfRevenuByServices["Implied Revenue"] / dfRevenuByServices["Implied Revenue"].sum()

# Get sum of implied revenue by zipcodes
dfRevenuByRevenue = df.groupby("Zip Code")["Implied Revenue"].sum().to_frame(name="Implied Revenue").reset_index()
dfRevenuByRevenue["Contribution"] = dfRevenuByRevenue["Implied Revenue"] / dfRevenuByRevenue["Implied Revenue"].sum()

In [14]:
dfRevenuBySegment.sort_values(by="Contribution", ascending=False)

,Segments,Implied Revenue,Contribution
0,Home Improvement,6596757.0,1.0


In [15]:
dfRevenuByServices.sort_values(by="Contribution", ascending=False)

,Services,Implied Revenue,Contribution
2,Hot Tub and Spa Repair,2750469.0,0.416943
1,Gutter Repair,1570004.0,0.237996
4,Lawn Mower Repair,1187134.0,0.179957
3,Insulation Installation or Upgrade,633311.0,0.096003
5,Roof Installation or Replacement,434815.0,0.065913
0,General Carpentry,21024.0,0.003187


In [16]:
dfRevenuByRevenue.sort_values(by="Contribution", ascending=False)

,Zip Code,Implied Revenue,Contribution
67,94108,402290.0,0.060983
7,20001,360864.0,0.054703
52,75207,344924.0,0.052287
68,95112,319992.0,0.048507
53,77002,306766.0,0.046503
...,...,...,...
36,57103,1620.0,0.000246
37,58102,1204.0,0.000183
38,58501,0.0,0.000000
39,59623,0.0,0.000000


In [17]:
df.to_csv("all_services.csv")